# Обучаем машины

В сериале "Силиконовая долина" один из героев сделал новое приложение.

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('mrk95jFVKqY')

Приложение бесполезное, но веселое и на нем можно просто показать полынй цикл обучения своей модели с нуля. Начнем.

In [ ]:
%matplotlib inline

Нам понадобится несколько библиотек

In [ ]:
import os  # используем для работы с файлами и папками
from random import random

import torch  # библиотека для нейронок
from IPython.display import clear_output  # функция удаляющая вывод чтобы не мусорить на экране.
from PIL import Image  # библиотека для работы с картинками
from matplotlib import pyplot as plt  # графики
from torch import nn, optim  # другие подмодули дял нейронок и их обучения
from torch.utils.data import Dataset, DataLoader  # другие вспомогательные модули torch-а
from torchvision import transforms, models  # вспомогательные функции для оьработки изображений
from tqdm import tqdm, trange  # библиотека дял отображения полоски загрузки

import warnings
warnings.filterwarnings('ignore')  # убирает лишние выводы с экрана

# PyTorch

Если вы не делаете сложных экспериментов, то обучать сетки могут даже дети.

In [ ]:
linear_model = nn.Linear(10, 1)

Вот так создается линейная модель в PyTorch.

In [ ]:
sequence_model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ELU(),
    nn.Linear(5, 10),
)

А вот так мы сделали модель, которая последовательно применяет два линейных слоя с активацией между ними.

Для обучения используются встроенные оптимизаторы

In [ ]:
optimizer = optim.SGD(sequence_model.parameters(), lr=1e-1)

Один шаг оптимизации:

1) Прогон модели от входа до результата

2) Высчитывание градиентов функции ошибки

3) Шаг (вычитание градиентов)

In [ ]:
input_tensor = torch.rand((1, 10), requires_grad=True)
real_answer = input_tensor * 2

In [ ]:
predictions = sequence_model(input_tensor)

In [ ]:
error = torch.mean(torch.abs(predictions - real_answer))
error

In [ ]:
error.backward()

In [ ]:
optimizer.step()

Теперь ошибка стала чуть меньше. Сделать так много раз - сеть обучится.

In [ ]:
error = torch.mean(torch.abs(sequence_model(input_tensor) - real_answer))
error

Теперь к задаче. Начнем с данных. Сейчас они предзагружены и лежат по папкам, пути до которых написаны ниже.

In [ ]:
DATA_FOLDER = '/workdir/data/'
IMAGES_FOLDER = os.path.join(DATA_FOLDER, 'images')

HOTDOGS_FOLDER = os.path.join(IMAGES_FOLDER, 'hotdogs')
HOTDOGS_FOLDER_TRAIN = os.path.join(HOTDOGS_FOLDER, 'train')
HOTDOGS_FOLDER_VAL = os.path.join(HOTDOGS_FOLDER, 'validation')
HOTDOGS_FOLDER_TEST = os.path.join(HOTDOGS_FOLDER, 'test')

BURGERS_FOLDER = os.path.join(IMAGES_FOLDER, 'burgers')
BURGERS_FOLDER_TRAIN = os.path.join(BURGERS_FOLDER, 'train')
BURGERS_FOLDER_VAL = os.path.join(BURGERS_FOLDER, 'validation')
BURGERS_FOLDER_TEST = os.path.join(BURGERS_FOLDER, 'test')

# hold out

Классическое разделение данных - train, validation, test. 

train - данные на которых модель учится

validation - данные на которые мы смотрим во вреям обучения, чтобы контролировать, как модель работает с картинками, на которых не обучалась. Сеть на них запускается, но не учится.

test - набор данных, на котором модель тестируется в самом конце. Он - финальный тест для нашей модели.

Немного кода для считывания картинок

In [ ]:
class FolderDataset:
    def __init__(self, folder, transform=None):
        self.files = [os.path.join(folder, image_path) for image_path in os.listdir(folder)]
        self.transform = transform
    
    def __getitem__(self, idx):
        result = Image.open(self.files[idx]).convert('RGB')
        
        if self.transform is None:
            return result
        else:
            return self.transform(result)
    
    def __len__(self):
        return len(self.files)

Выглядит сложно, но на самом деле это просто класс, который можно попросить загрузить картинку под каким-то номером.

Посмотрим на пару примеров из наших данных.

In [ ]:
hotdogs_images = FolderDataset(HOTDOGS_FOLDER_TRAIN)

In [ ]:
hotdogs_images[1]

In [ ]:
burgers_images = FolderDataset(BURGERS_FOLDER_TRAIN)

In [ ]:
burgers_images[1]

Я просто взял и скачал по 200 картинок из гугла по запросу burger и hotdog.

А теперь напишем нашу сеть

In [ ]:
class HotdogClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(  # будет вызывать поочереди все переданные слои
            nn.Conv2d(3, 16, (3, 3)),  # 256 x 256 x 3 -> 256 x 256 x 16
            nn.LeakyReLU(),
            nn.Conv2d(16, 32, (5, 5), stride=2, padding=2),  # 256 x 256 x 16 -> 128 x 128 x 32
            nn.LeakyReLU(),
            nn.Conv2d(32, 64, (5, 5), stride=2, padding=2),  # 128 x 128 x 32 -> 64 x 64 x 64
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, (5, 5), stride=2, padding=2),  # 64 x 64 x 64 -> 32 x 32 x 128
            nn.LeakyReLU(),
            nn.Conv2d(128, 256, (5, 5), stride=2, padding=2),  # 32 x 32 x 128 -> 16 x 16 x 256
            nn.LeakyReLU(),
        )
        self.fc = nn.Linear(256 * 8 * 8, 1)  # 16 x 16 x 256 -> 1
    
    def forward(self, x):  # функция получающая вход и выдающая результат
        x = self.seq(x)
        x = x.view(-1, 256 * 8 * 8)
        return self.fc(x)

In [ ]:
def exponential_smoothing(values, alpha=0.8):  # дополнительная функция, чтобы графики выводились более плавно
    smoothed_values = [values[0]]
    for x in values[1:]:
        smoothed_values.append(smoothed_values[-1] * alpha + x * (1 - alpha))
    
    return smoothed_values

Напишем функцию для обучения

In [ ]:
def train(model, optimizer, hotdogs_train, hotdogs_val, burgers_train, burgers_val, epoch_count=5):
    batch_size = 4
    
    # подготавливаем загрузку данных в нужном формате
    hotdogs_train_loader = DataLoader(hotdogs_train, batch_size=batch_size, shuffle=True, drop_last=True)
    burgers_train_loader = DataLoader(burgers_train, batch_size=batch_size, shuffle=True, drop_last=True)
    
    mse_loss = nn.MSELoss()  # функция ошибки - то что мы хотим уменьшать. Цель нашей модели.
    
    log = []
    log_val = []
    
    for epoch_num in range(epoch_count):
        title = 'Epoch {}/{}'.format(epoch_num + 1, epoch_count)
        for step_num, (hotdog_tensor, burger_tensor) in tqdm(enumerate(zip(hotdogs_train_loader, burgers_train_loader)), desc=title, total=len(hotdogs_train_loader)):
            optimizer.zero_grad()
            
            # получаем предсказания
            predictions = nn.functional.sigmoid(model(torch.cat([hotdog_tensor, burger_tensor])))
            #  получаем настоящие ответы, где хотдоги, а где нет
            answers = torch.cat([torch.ones(batch_size), torch.zeros(batch_size)])[..., None]

            loss = mse_loss(predictions, answers)  # вычисляем ошибку
            loss.backward()

            optimizer.step()  # пытаемся ее уменьшить
            
            # рисуем гарфики ошибок
            if step_num % 2 == 0:
                hotdogs_val_loader = DataLoader(hotdogs_val, batch_size=batch_size, shuffle=True)
                burgers_val_loader = DataLoader(burgers_val, batch_size=batch_size, shuffle=True)

                val_predictions = nn.functional.sigmoid(model(torch.cat([next(iter(hotdogs_val_loader)), next(iter(burgers_val_loader))])))
                val_answers = torch.cat([torch.ones(batch_size), torch.zeros(batch_size)])[..., None]

                loss_val = mse_loss(val_predictions, val_answers)

                log.append(loss.detach_())
                log_val.append(loss_val.detach_())
                
                clear_output()
                plt.plot(exponential_smoothing(log), color='blue', label='training loss')
                plt.plot(exponential_smoothing(log_val), color='red', label='validation loss')
                plt.legend()
                plt.show()

И, конечно, функция для финального теста. Просто запускаем нашу модель на всех тестовых изображениях и считаем долю правильных ответов.

In [ ]:
def test_model(model, hotdogs_test, burgers_test):
    hotdogs_test_loader = DataLoader(hotdogs_test, batch_size=1)
    burgers_test_loader = DataLoader(burgers_test, batch_size=1)
    
    log = []
    total = len(hotdogs_test_loader)
    
    for hotdog_tensor, burger_tensor in tqdm(zip(hotdogs_test_loader, burgers_test_loader), total=total):
        predictions = model(torch.cat([hotdog_tensor, burger_tensor]))
        answers = torch.cat([torch.ones(1), torch.zeros(1)])[..., None]

        is_hotdog = predictions > 0
        accuracy = is_hotdog.type(torch.int8) == answers.type(torch.int8)
        log.append(accuracy)
            
    return torch.cat(log).type(torch.float32).mean()

Ставим модель обучаться

In [ ]:
transform_resize = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])  # меняем размер данных и превращаем в тензор - способ представления изображений для сеток.

# применяем трансформацию к датасетам
hotdogs_train = FolderDataset(HOTDOGS_FOLDER_TRAIN, transform_resize)
hotdogs_val = FolderDataset(HOTDOGS_FOLDER_VAL, transform_resize)
hotdogs_test = FolderDataset(HOTDOGS_FOLDER_TEST, transform_resize)

burgers_train = FolderDataset(BURGERS_FOLDER_TRAIN, transform_resize)
burgers_val = FolderDataset(BURGERS_FOLDER_VAL, transform_resize)
burgers_test = FolderDataset(BURGERS_FOLDER_TEST, transform_resize)

In [ ]:
hotdog_classifier = HotdogClassifier()  # создаем экземляр сетки

Посмотрим на ее качество, на момент, когда оан еще не обучалась.

In [ ]:
test_model(hotdog_classifier, hotdogs_test, burgers_test)

In [ ]:
optimizer = optim.Adam(hotdog_classifier.parameters(), lr=1e-4, amsgrad=True)  # оптимизатор - написанный за нас способ оптимизации весов нашей сети

Поставим обучаться сеть.

* Можно задать вопросы.
* Можно передохнуть (так желают все ресерчеры)
* Что такое loss?
* Почему он падает?
* Как связаны loss валидации и обучения?
* Как выбираюсь архитектуру сети?
* Как работают с данными?

In [ ]:
train(hotdog_classifier, optimizer, hotdogs_train, hotdogs_val, burgers_train, burgers_val, epoch_count=20)

In [ ]:
test_model(hotdog_classifier, hotdogs_test, burgers_test)

Получили результат обученной сети. К слову, она переобучилась! Это когда модель слишком зациклена на данных, котоыре она видела и не смотрит на общую картину. Она просто запомнила данные для обучения, но не обнаружила достаточно много паттернов.

Попробуем отражать картинки по горизонтали и немного портить их. Это называется аугментацией. Данных у нас очень мало. Хотя кажется, что порча картинок ухудшит качество, в реальности так делают очень часто. Делов . том что мы таким образом как бы "бесплатно" создаем новые картинки. Больше датасет - лучше обучение.

In [ ]:
def gaussian(x, mean=0, std=0.05, p=0.1
            ):
    if random() < p:
        noise = x.clone().normal_(mean, std)
        x = (x + noise).clamp(0, 1)
    return x

In [ ]:
transform_augment = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    gaussian,
])

hotdogs_train = FolderDataset(HOTDOGS_FOLDER_TRAIN, transform_augment)
hotdogs_train = FolderDataset(HOTDOGS_FOLDER_TRAIN, transform_augment)

In [ ]:
transforms.functional.to_pil_image(hotdogs_train[0])

In [ ]:
hotdog_classifier_with_flip = HotdogClassifier()

In [ ]:
optimizer = optim.Adam(hotdog_classifier_with_flip.parameters(), lr=1e-4, amsgrad=True)

In [ ]:
train(hotdog_classifier_with_flip, optimizer, hotdogs_train, hotdogs_val, burgers_train, burgers_val, epoch_count=20)

In [ ]:
test_model(hotdog_classifier_with_flip, hotdogs_test, burgers_test)

In [ ]:
hotdog_classifier_with_flip

In [ ]:
import PIL

def show_featuremap(image, model, layer, image_size=128):
    tensor = hotdogs_test[0]
    transforms.functional.to_pil_image(tensor)
    featuremap_maker = list(hotdog_classifier_with_flip.children())[0][:layer + 1]
    feature_map = featuremap_maker(tensor[None, ...])
        
    big_image = Image.new('RGB', (4 * image_size, (feature_map.size(1) // 4) * image_size))
    
    for feature_num in range(feature_map.size(1)):
        row_num = feature_num % 4
        col_num = feature_num // 4
        
        small_image = feature_map[:, feature_num]
        big_image.paste(
            transforms.functional.resize(
                transforms.functional.to_pil_image(
                    feature_map[:, feature_num]
                ),
                (image_size, image_size),
                interpolation=0
            ),
            (row_num * (image_size), col_num * (image_size))
        )
    
    return big_image

Промежуточные представления картинок

In [ ]:
show_featuremap(hotdogs_test, hotdog_classifier_with_flip, 9)

Вторая модель либо лучше, либо хуже. Ресерч - это в первую очередь эксперименты. Иногда идеи просто не работают.

По сути мы прошли полный цикл создания простейшей модели: от скачивания данных до обученной сети. Естественно, расчитывать на идеальное качество за такое время не приходится, но, на минуточку, мы только что воспроизвели сеть из одного из самых популярных в мире сериалов про технарей!

Попробуйте подергать разные настройки или может быть у вас есть какая-то идея, но вы не знаете как ее написать на питоне. Скажите мне и мы попробуем поэкспериментировать.